In [2]:
from keras.layers import Input, Dense, LSTM, RepeatVector
from keras.models import Model
from keras.datasets import mnist
import numpy as np
# import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import time
from keras_tqdm import TQDMNotebookCallback



In [2]:
import seq2seq
from seq2seq.models import Seq2Seq, SimpleSeq2Seq


ImportError: cannot import name 'Layer'

In [12]:
import keras
keras.__version__

AttributeError: module 'keras' has no attribute '__version__'

In [9]:
dir(keras)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'layers']

In [20]:

model = Seq2Seq(batch_input_shape=(16, 7, 5), hidden_dim=10, output_length=8, output_dim=20, depth=4, peek=True)
model.compile(loss='mse', optimizer='rmsprop')

TypeError: __call__() got an unexpected keyword argument 'name'

In [6]:
word2vec_model = KeyedVectors.load_word2vec_format('lemmas.cbow.s100.w2v.bin', binary=True)

In [7]:
word2vec_model

In [8]:
with open('tee_both_contexts_s_False_w_2.txt') as f:
    lines = f.readlines()

symmetric_lines = [line for line in lines if len(line.split())==4]
lines = symmetric_lines

X = np.zeros((len(lines), 400))
for i, line in enumerate(lines):
    X[i] = word2vec_model[line.split()].flatten()    

x_train, x_test = train_test_split(X, test_size=0.1, random_state=1)

In [19]:
# RECURRENT
X = np.zeros((len(lines), 4, 100))
for i, line in enumerate(lines):
    X[i] = word2vec_model[line.split()]

x_train, x_test = train_test_split(X, test_size=0.1, random_state=1)

In [34]:
def train(latent_dim, batch_size):

    timesteps = 4
    input_dim = 100
#     latent_dim = 32
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dim)(inputs)

    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)

    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)

    sequence_autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')

    model_number = int(time.time()-1491000000)
    checkpointer = ModelCheckpoint(filepath="model-weights/{}.hdf5".format(model_number), verbose=0, save_best_only=True)
    json_string = sequence_autoencoder.to_json()
    with open('model-architectures/{}.json'.format(model_number), 'w') as f:
        f.write(json_string)

    sequence_autoencoder.fit(x_train, x_train,
                    epochs=200,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, x_test),
                    verbose=0,
                    callbacks=[TensorBoard(log_dir='training-logs/{}'.format(model_number), write_graph=False),
                                EarlyStopping(verbose=0, patience=10),
                                checkpointer,
                                TQDMNotebookCallback(leave_inner=False)
                              ])

In [41]:
train(latent_dim=128, batch_size=64)

In [39]:
train(latent_dim=128, batch_size=64)

In [40]:
train(latent_dim=128, batch_size=256)

In [28]:
train(16, batch_size=64)

In [29]:
train(64, batch_size=64)

In [30]:
train(128, batch_size=64)

In [31]:
train(256, batch_size=64)

In [32]:
train(512, batch_size=64)

In [35]:
train(128, batch_size=64)

In [36]:
train(128, batch_size=64)

In [37]:
train(128, batch_size=64)

In [38]:
train(128, batch_size=64)